### IMDB dataset having 50K movie reviews for natural language processing or Text analytics.
### This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets

SOURCE: Kaggle
link: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [0]:
import spacy
import nltk
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from spacy import lemmatizer
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np
from google.colab import drive

In [4]:
# tf.enable_eager_execution()
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
nlp=spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'ner'])

In [0]:
tokenizer=RegexpTokenizer("(\w+\'?\w+?)")

In [7]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
sw1=stopwords.words("english")
sw2=STOP_WORDS
stop_words=set(sw1).union(sw2)

In [0]:
def tokenize(rev):
    return(tokenizer.tokenize(str(rev).lower()))

In [0]:
def remove_stop_words(rev_tokens):
    return([tok for tok in rev_tokens if tok not in stop_words])

In [0]:
def lemmatize(rev_tokens):
    result=[]
    for tok in rev_tokens:
        temp=nlp(tok)
        for tok in temp:
            result.append(tok.lemma_)
    return result

In [0]:
def preprocess_pipeline(review):
    review=tokenize(review)
    review=remove_stop_words(review)
    review=lemmatize(review)
    return review

In [13]:
df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/IMDB Dataset.csv')
df.head(2)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive


In [0]:
reviews=list(df['review'])
sentiments=list(df['sentiment'])

In [0]:
reviews=list(map(lambda x: preprocess_pipeline(x), reviews))

##### word embddings using word2vec from gensim

In [0]:
dimension=100 # will also be used in the RNN unit part 
model=Word2Vec(reviews, size=dimension, window=3, min_count=3, workers=4)

In [17]:
model.sg#its CBOW not skipgram

0

In [0]:
# tip: as soon as we train our model, we have to delete the model unless there is further training or updation required in the model because it consumes lots of memory
# but, to use it even after deleting the model, we can use key'd vector model that holds all the info about the embedding model
word_vec=model.wv
del(model)

In [19]:
#save the vocabulary of the model
len(word_vec.vocab)

38859

In [20]:
word_vec.similar_by_word(word="bad", topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('terrible', 0.747195303440094),
 ('horrible', 0.741730809211731),
 ('awful', 0.7321454286575317),
 ('good', 0.7019721865653992),
 ('lousy', 0.6956417560577393),
 ('suck', 0.6857665777206421),
 ('crappy', 0.64302659034729),
 ('atrocious', 0.6322519183158875),
 ('lame', 0.6303330659866333),
 ('mediocre', 0.6185406446456909)]

In [21]:
word_vec.similarity("good", "be")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.13662036

In [22]:
#now apply contextual relation with the word
#ex: king - man + woman = queen
#example: 
word_vec.most_similar(negative=["bad"], positive=["decent"], topn=5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('modest', 0.4140951633453369),
 ('solid', 0.4094703495502472),
 ('fine', 0.3787546157836914),
 ('creditable', 0.3685643672943115),
 ('workmanlike', 0.3478650748729706)]

In [0]:
# from gensim.models import KeyedVectors #to store the loaded pretrained model

In [0]:
# model=KeyedVectors.load_word2vec_format('pretrained embedding path in .bin format', binary=True)

### Build the model for sentiment analysis using RNN

In [0]:
#to ensure results are reproducable set these
SEED=2031
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic=True

Take in the text in the review and convert into corresponding index (dictionary) for this dictionary, key'd vector from gensim is used. Using that dictionary we'd convert all the words in our text corpus as indices and then we pass those indices through the RNN

In [0]:
def convWordInd(embedding_model, review):
    indice_rev=[]
    for word in review:
        try:
            indice_rev.append(embedding_model.vocab[word].index)
        except:
            pass
    return torch.tensor(indice_rev)

In [0]:
review_indexes=list(map(lambda x: convWordInd(word_vec, x), reviews))

Passing the data to nn as batches. When it is done so, all the sentance should have the same length For that, all the statements are padded with the padding value (len of longest statement in the batch). So, the padding value is appended to the end of the shorter statements (that is the index value that's why it is done so)

In [28]:
pad_value=len(word_vec.index2word) #used later during RNN param initialization
pad_value #this is the length of the longest review in the batch

38859

#### BUILD THE RNN MODEL

layers:
1. Embedding layer - input - indexed reviews and convert to embedded format 
2. RNN unit - input - embedded representation
3. Fully connected unit (dense)

Embedding layer :Word embeddings are already created using gensim and these pre-trained embeddings are used within our nn. For that first extract the weights that the gensim model has learned while training

RNN Model-for training and evaluation.
After that the  dense layer is declared

forward function: for data when passed onto the model, for each batch x, with size (max(len(sentances)*len(batch)))

pytorch internally converts the indexed representation which we passed to one hot encoding.

##### NOTE
In the forward pass function, batch of reviews and the len of each review (text length) is passed. But, in case of RNN, each and every review must have same length. However, it is not required that the model to read that padded values. Hence we use the pytorch inbuilt function called "packpaddedsequence". It packs the padded values automatically and internally handles for us. 

So the packed sequence is passed through the RNN and after passing through the RNN, the packed sequence is unlocked. 

In [0]:
embed_weights=torch.Tensor(word_vec.vectors)

In [0]:
class RNN(nn.Module):
    def __init__(self, inp_dim, embed_dim, hidden_dim, out_dim, n_layers, bidirectional, dropout, embed_weights):
        super().__init__()
        self.embedding_layer=nn.Embedding.from_pretrained(embed_weights)
        self.rnn=nn.LSTM(embed_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.dense=nn.Linear(hidden_dim*2, out_dim)#ip layer is hid_layer*2 because in case of bidirectional RNN, there are 2 hidden layer o/p
        self.dropout=nn.Dropout(dropout)#never use dropout in the ip or op layers but in the intermediate layers
        
    def forward(self, x, text_lens):
        embedded=self.embedding_layer(x)
        packed_embed=nn.utils.rnn.pack_padded_sequence(embedded, text_lens)
        packed_out, (hidden, cell)=self.rnn(packed_embed) #output size=[text_len, batch_size, hiddendim*num of dim]
        #output, output_lens=nn.utils.rnn.pad_packed_sequence(packed_out) commented as output is not used here 
        #bdirlstm consists [f0, b0, f1, b1, ..... fn, bn]
        #so, concatinating the last two hidden state (forward and backward) from the last layer, it is passed to linear layer 
        hidden =self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        return self.dense(hidden.squeeze(0))        

Define the parameters for the  RNN class and create the objects of the  RNN Class

In [0]:
#updating the hyperparameters
inp_dim=pad_value
embed_dim=dimension
hidden_dim=256
output_dim=1
n_layers=2
bidirectional=True
dropout=0.5

In [32]:
model=RNN(inp_dim, embed_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, embed_weights)
model

RNN(
  (embedding_layer): Embedding(38859, 100)
  (rnn): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (dense): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [0]:
optimizer=optim.Adam(model.parameters())
loss_function=nn.BCEWithLogitsLoss()

In [34]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#### prepare the data for :

    a. Training

    b. Validation
    
    c. Testing

In [35]:
#binary encoding the y vals
sentiments=[0 if label == 'negative' else 1 for label in sentiments]
sentiments[:5]

[1, 1, 1, 0, 1]

In [0]:
X_train, X_test, Y_train, Y_test=train_test_split(review_indexes, sentiments, test_size=0.25)
X_train, X_val, Y_train, Y_val=train_test_split(X_train, Y_train, test_size=0.2)

In [0]:
#now batches out of entire dataset is prepared
batch_size=128
def iterate_func(x, y):
    size=len(x)
    permute=np.random.permutation(size) #for creating the random blocks from the list of reviews
    iterator=[]
    for i in range(0, size, batch_size): #from 0 to size stepping from batch size
        indices=permute[i: i+batch_size]
        batch={}
        batch["text"]=[x[i] for i in indices]
        batch["label"]=[y[i] for i in indices]
        #sort the texts based on their lengths
        batch["text"], batch["label"]=zip(*sorted(zip(batch["text"], batch["label"]), key=lambda x: len(x[0]), reverse=True))
        batch["length"]=[len(rev) for rev in batch["text"]]
        batch["length"]=torch.IntTensor(batch["length"])
        #Now, reviews are padded in each batch and are passed into the model. 
        #For padding, pytorch offers the method within its nn module
        batch["text"]=torch.nn.utils.rnn.pad_sequence(batch["text"], batch_first=True).t()#transpose is performed so that it is accepted into rnn
        batch["label"]=torch.Tensor(batch["label"])
        
        #now pushing all to the gpu
        batch["text"]=batch["text"].to(device)
        batch["label"]=batch["label"].to(device)
        batch["length"]=batch["length"].to(device)
        
        iterator.append(batch)
        
    return iterator

In [0]:
train_iter=iterate_func(X_train, Y_train)
val_iter=iterate_func(X_val, Y_val)
test_iter=iterate_func(X_test, Y_test)

In [0]:
model=model.to(device)
criterion=loss_function.to(device)

#### defining a function to know the accuracy subsequently

In [0]:
def binary_acc(preds, y):
    round_preds=torch.round(torch.sigmoid(preds))
    pos=(round_preds==y).float()
    accuracy=pos.sum()/len(pos)
    return accuracy

#### defining the function to train the model 
set the loss and accuracies as false from the previous use and set the model to training mode

In [0]:
def train(model, iterator, optimizer, criterion):
    epoch_loss=0
    epoch_accuracy=0
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions=model(batch["text"], batch["length"]).squeeze(1)
        loss=criterion(predictions, batch["label"])
        accuracy=binary_acc(predictions, batch["label"])
        loss.backward()
        optimizer.step() #updates the weight for the model
        epoch_loss+=loss.item()
        epoch_accuracy+=accuracy.item()
    
    #return the average epoch loss and iterator
    return(epoch_loss/len(iterator), epoch_accuracy/len(iterator))

#### defining the function to Test the model
set the loss and accuracies as false from the previous use and set the model to evaluation mode

In [0]:
def evaluator(model, iterator, criterion):
    epoch_loss=0
    epoch_accuracy=0
    model.eval()
    #to prevent any gradient calculation with nograd is used
    with torch.no_grad():
        for batch in iterator:
            predictions=model(batch["text"], batch["length"]).squeeze(1)
            loss=criterion(predictions, batch["label"])
            accuracy=binary_acc(predictions, batch["label"])
            epoch_loss+=loss.item()
            epoch_accuracy+=accuracy.item()

    #return the average epoch loss and iterator
    return(epoch_loss/len(iterator), epoch_accuracy/len(iterator))

#### Running the model and evaluation 

In [43]:
epochs=10

for epoch in range(epochs):
    train_loss, train_accuracy=train(model, train_iter, optimizer, criterion)
    valid_loss, valid_accuracy=evaluator(model, val_iter, criterion)
    
    print("Epoch number: ", epoch)
    print("Train Loss = ", train_loss, " Train Accuracy = ", train_accuracy)
    print("Validation Loss = ", valid_loss, " Validation Accuracy = ", valid_accuracy)

Epoch number:  0
Train Loss =  0.46462931937359747  Train Accuracy =  0.7737588654173181
Validation Loss =  0.5330934958942866  Validation Accuracy =  0.724966548256955
Epoch number:  1
Train Loss =  0.3770504614774217  Train Accuracy =  0.8346520390916378
Validation Loss =  0.346875739804769  Validation Accuracy =  0.8596955840870485
Epoch number:  2
Train Loss =  0.32868919867150326  Train Accuracy =  0.8608599292471054
Validation Loss =  0.3099044265888505  Validation Accuracy =  0.8707418047775657
Epoch number:  3
Train Loss =  0.29549820854308756  Train Accuracy =  0.8771054965384463
Validation Loss =  0.28767729891558824  Validation Accuracy =  0.8827428080267825
Epoch number:  4
Train Loss =  0.2856949260894288  Train Accuracy =  0.8813054079705096
Validation Loss =  0.29537911738379524  Validation Accuracy =  0.8773555977869842
Epoch number:  5
Train Loss =  0.26939209439653033  Train Accuracy =  0.8893949468085106
Validation Loss =  0.27792145160295195  Validation Accuracy =  

In [44]:
test_loss, test_accuracy=evaluator(model, test_iter, criterion)
print("Epoch number: ", epoch)
print("Test Loss = ", test_loss, " Test Accuracy = ", test_accuracy)

Epoch number:  9
Test Loss =  0.3236091924565179  Test Accuracy =  0.8800450983096142
